In [ ]:
import os
import csv
import csv
import os
import pandas as pd
import unicodedata
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

import numpy as np

In [ ]:
def _extract_text():

    fields = ["Sentence", "Emotion", "Session", "Session"]
    with open("text_data.csv", "a") as f:
        write = csv.writer(f)
        write.writerow(fields)

    for i in range(1, 6):
        directory = f"/content/drive/MyDrive/Dataset/Session{i}/dialog/transcriptions"
        directory1 = f"/content/drive/MyDrive/Dataset/Session{i}/dialog/EmoEvaluation"
        print(directory)
        emotions = []

        final = []
        transcripts = []
        ses_list_final = []
        emo_ses_list = []

        transcriptions = os.listdir(directory)
        transcriptions.sort()

        emoEvaluation = os.listdir(directory1)
        emoEvaluation.sort()

        # print("hi")
        for filename1 in emoEvaluation:
            file_path = directory1 + "/" + filename1
            # print(file_path)
            file = open(file_path, "r")
            lines = file.readlines()

            for line in lines:
                # print(line)
                if line[0] == "[":
                    emotions.append(line.split()[4])
                    emo_ses_list.append(line.split()[3])

        for filename in transcriptions:
            # print(filename)
            m_list = []
            f_list = []
            ses_list_f = []
            ses_list_m = []

            file_path = directory + "/" + filename
            file = open(file_path, "r")
            transcript_lines = file.readlines()

            for lines in transcript_lines:
                lines = lines.split(":")

                if len(lines[0]) > 10:

                    if lines[0][-24] == "F" and lines[0][-23] != "X":
                        f_list.append(lines[1][1:-1])
                        ses_list_f.append(lines[0])

                    elif lines[0][-24] == "M" and lines[0][-23] != "X":
                        m_list.append(lines[1][1:-1])
                        ses_list_m.append(lines[0])

            transcripts += f_list + m_list
            ses_list_final += ses_list_f + ses_list_m

        for i in range(len(transcripts)):
            final.append([transcripts[i], emotions[i],
                        ses_list_final[i], emo_ses_list[i]])

        rows = final

        with open("text_data.csv", "a") as f:
            write = csv.writer(f)
            write.writerows(rows)


In [ ]:
df=pd.read_csv('metadata.csv')

In [ ]:
df.head()

,session,method,gender,emotion,n_annotators,agreement,path,file_name
0,1,impro,F,neu,4,4,Session1/sentences/wav/Ses01F_impro01/Ses01F_i...,Ses01F_impro01_F000
1,1,impro,F,neu,4,4,Session1/sentences/wav/Ses01F_impro01/Ses01F_i...,Ses01F_impro01_F001
2,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro01/Ses01F_i...,Ses01F_impro01_F002
3,1,impro,F,xxx,0,0,Session1/sentences/wav/Ses01F_impro01/Ses01F_i...,Ses01F_impro01_F003
4,1,impro,F,xxx,0,0,Session1/sentences/wav/Ses01F_impro01/Ses01F_i...,Ses01F_impro01_F004


In [ ]:
def _unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [ ]:
# Lowercase, trim, and remove non-letter characters
def _normalizeString(s):
    s = _unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def make_text_features():
    _extract_text()

    data = pd.read_csv("text_data.csv")
    transcripts = data['Sentence'].tolist()

    for i in range(len(transcripts)):

        transcripts[i] = transcripts[i].replace("'", "")
        transcripts[i] = _normalizeString(transcripts[i])


    data["Cleaned"] = transcripts

    data.to_csv('text_data_clean.csv', index=False)

In [ ]:
make_text_features()

/content/drive/MyDrive/Dataset/Session1/dialog/transcriptions
/content/drive/MyDrive/Dataset/Session2/dialog/transcriptions
/content/drive/MyDrive/Dataset/Session3/dialog/transcriptions
/content/drive/MyDrive/Dataset/Session4/dialog/transcriptions
/content/drive/MyDrive/Dataset/Session5/dialog/transcriptions


In [ ]:
def get_data():

    df = pd.read_csv('text_data_clean.csv')
    df.drop(df.loc[(df['Emotion'] == 'xxx') | (df['Emotion'] == 'dis') | (df['Emotion'] == 'oth') | (df['Emotion'] == 'fea') | (df['Emotion'] == 'sur') ].index, inplace = True)
    df.loc[(df['Emotion'] == 'exc'), 'Emotion'] = 'hap'

    df = shuffle(df, random_state=42)

    x = df["Cleaned"]
    y = df["Emotion"]
    # y = df["Emotion"].map({'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3})
    encoder = LabelEncoder()
    y = encoder.fit_transform(y)

    return x, y


In [ ]:
def get_train_test():
    x, y = get_data()
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)

    cv = CountVectorizer(min_df=5, ngram_range=(1, 2))
    x_train = cv.fit_transform(x_train)
    x_test = cv.transform(x_test)

    return x_train, x_test, y_train, y_test

In [ ]:
def get_logistic_regression():
    return LogisticRegression(solver='liblinear', random_state=42)

def main():
    lr = get_logistic_regression()
    x_train, x_test, y_train, y_test = get_train_test()
    # print(x_train[:5])
    lr.fit(x_train, y_train)

    results = lr.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 4))), ('estimator', get_logistic_regression())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))



In [ ]:
print('On Train/Test Split', end='\n\n')
main()
print('On K-fold Cross Validation', end='\n\n')
k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           1     0.5938    0.3654    0.4524        52
           2     0.4000    0.3548    0.3761        62
           3     0.4375    0.4286    0.4330        49
           4     0.4175    0.5890    0.4886        73
           5     0.3889    0.3684    0.3784        38

    accuracy                         0.4343       274
   macro avg     0.4475    0.4213    0.4257       274
weighted avg     0.4466    0.4343    0.4310       274

On K-fold Cross Validation



/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Accuracy:  0.4513542418651908
F1 Macro:  0.4078266865842271
Precision Macro:  0.4356777826014036
Recall Macro:  0.4011382475901927
F1 Weighted:  0.4457889279785987
Precision Weighted:  0.4637322556361066
Recall Weighted:  0.4513542418651908


In [ ]:
def get_random_forest():
    # return RandomForestClassifier(random_state=42)
    return RandomForestClassifier(**{
        'random_state': 42,
        'class_weight': None,
        'criterion': 'gini',
        'max_depth': 2000,
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'n_estimators': 1500
    })

def main():
    rfc = get_random_forest()
    x_train, x_test, y_train, y_test = get_train_test()
    rfc.fit(x_train, y_train)

    results = rfc.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_random_forest())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           1     0.5405    0.3846    0.4494        52
           2     0.4545    0.4032    0.4274        62
           3     0.5600    0.5714    0.5657        49
           4     0.4479    0.5890    0.5089        73
           5     0.5000    0.4737    0.4865        38

    accuracy                         0.4891       274
   macro avg     0.5006    0.4844    0.4876       274
weighted avg     0.4943    0.4891    0.4862       274

On K-fold Cross Validation



/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Accuracy:  0.47400069516857846
F1 Macro:  0.4309086511430256
Precision Macro:  0.45333878444290737
Recall Macro:  0.42405573729601187
F1 Weighted:  0.4702378376407168
Precision Weighted:  0.4829530562446044
Recall Weighted:  0.47400069516857846


In [ ]:
def get_xgb_classifier():
    return xgb.XGBClassifier(**{
        'random_state': 42,
        'n_jobs': -1,
        'max_depth': 6,
        'n_estimators': 200,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'objective': 'multi:softmax',
        'num_class': 3
    })
def get_xgb():
    # return XGBClassifier(random_state=42, tree_method='gpu_hist')
    return XGBClassifier(**{
        'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'reg_lambda': 0.001, 'subsample': 0.4, 'random_state': 42
    })

def main():
    xgb_classifier = get_xgb_classifier()
    # xgb_classifier = get_xgb()
    x_train, x_test, y_train, y_test = get_train_test()
    print()
    xgb_classifier.fit(x_train, y_train)

    results = xgb_classifier.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_xgb_classifier())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()


On Train/Test Split


              precision    recall  f1-score   support

           0     0.7486    0.6298    0.6841       208
           1     0.6655    0.6088    0.6359       317
           2     0.5646    0.6992    0.6247       369
           3     0.6162    0.5352    0.5729       213

    accuracy                         0.6287      1107
   macro avg     0.6487    0.6183    0.6294      1107
weighted avg     0.6380    0.6287    0.6291      1107

On K-fold Cross Validation

Accuracy:  0.6414748493476496
F1 Macro:  0.6421945325700424
Precision Macro:  0.6748521122146427
Recall Macro:  0.6283877270174832
F1 Weighted:  0.6426804866305873
Precision Weighted:  0.662491227868814
Recall Weighted:  0.6414748493476496


In [ ]:
def get_xgb():
    # return XGBClassifier(random_state=42, tree_method='gpu_hist')
    return XGBClassifier(**{
        'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'reg_lambda': 0.001, 'subsample': 0.4, 'random_state': 42
    })

def main():
    xgb = get_xgb()
    x_train, x_test, y_train, y_test = get_train_test()
    # print(x_train,x_test,y_train,y_test)
    xgb.fit(x_train, y_train)

    results = xgb.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_xgb())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           0     0.7557    0.6394    0.6927       208
           1     0.6801    0.6372    0.6580       317
           2     0.5829    0.6856    0.6301       369
           3     0.5900    0.5540    0.5714       213

    accuracy                         0.6378      1107
   macro avg     0.6522    0.6291    0.6381      1107
weighted avg     0.6446    0.6378    0.6386      1107

On K-fold Cross Validation

Accuracy:  0.6445478469251239
F1 Macro:  0.6469443298212532
Precision Macro:  0.6667118107410428
Recall Macro:  0.6362928928873026
F1 Weighted:  0.6462905842841389
Precision Weighted:  0.657086982870736
Recall Weighted:  0.6445478469251239


In [ ]:
def get_naive_bayes():
    # return MultinomialNB()
    return MultinomialNB(alpha=0.275)

def main():
    mnb = get_naive_bayes()
    x_train, x_test, y_train, y_test = get_train_test()
    mnb.fit(x_train, y_train)

    results = mnb.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_naive_bayes())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           0     0.7312    0.6538    0.6904       208
           1     0.6109    0.5994    0.6051       317
           2     0.5601    0.6314    0.5936       369
           3     0.5928    0.5399    0.5651       213

    accuracy                         0.6089      1107
   macro avg     0.6237    0.6061    0.6135      1107
weighted avg     0.6131    0.6089    0.6096      1107

On K-fold Cross Validation

Accuracy:  0.645813506356884
F1 Macro:  0.6463176522593687
Precision Macro:  0.6625101207606496
Recall Macro:  0.6376181370200869
F1 Weighted:  0.6461811538130076
Precision Weighted:  0.6533677235685771
Recall Weighted:  0.645813506356884


In [ ]:
def get_svm():
    # return SVC(kernel='linear', probability=True, random_state=42)
    return SVC(kernel='rbf', gamma='scale', C=1.85, probability=True, random_state=42)

def main():
    svm = get_svm()
    x_train, x_test, y_train, y_test = get_train_test()
    svm.fit(x_train, y_train)

    results = svm.predict(x_test)
    print(classification_report(y_test, results, digits=4))

def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_svm())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro',
           'f1_weighted' : 'f1_weighted',
           'precision_weighted' : 'precision_weighted',
           'recall_weighted' : 'recall_weighted'
           }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))
    print('F1 Weighted: ', np.mean(scores['test_f1_weighted']))
    print('Precision Weighted: ', np.mean(scores['test_precision_weighted']))
    print('Recall Weighted: ', np.mean(scores['test_recall_weighted']))

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           0     0.8322    0.5962    0.6947       208
           1     0.6796    0.6625    0.6709       317
           2     0.5644    0.7480    0.6434       369
           3     0.6562    0.4930    0.5630       213

    accuracy                         0.6459      1107
   macro avg     0.6831    0.6249    0.6430      1107
weighted avg     0.6654    0.6459    0.6454      1107

On K-fold Cross Validation

Accuracy:  0.6423788451266068
F1 Macro:  0.6428044822540336
Precision Macro:  0.6855739502744447
Recall Macro:  0.6260786644152045
F1 Weighted:  0.6432842885293597
Precision Weighted:  0.6685545150579009
Recall Weighted:  0.6423788451266068


In [ ]:
def get_mlp():
    # return MLPClassifier(random_state=42, max_iter=5000)
    return MLPClassifier(
        random_state=42, activation='relu',
        alpha=0.35,hidden_layer_sizes=(150,),
        learning_rate='adaptive', solver='adam')

def main():
    mlp = get_mlp()
    x_train, x_test, y_train, y_test = get_train_test()
    mlp.fit(x_train, y_train)

    results = mlp.predict(x_test)
    report = classification_report(y_test, results, digits=4, output_dict=True)
    print(classification_report(y_test, results, digits=4))

    f1_weighted = report['weighted avg']['f1-score']
    precision_weighted = report['weighted avg']['precision']
    recall_weighted = report['weighted avg']['recall']
    print('F1 Weighted: ', f1_weighted)
    print('Precision Weighted: ', precision_weighted)
    print('Recall Weighted: ', recall_weighted)
def k_fold():
    pipeline = Pipeline(
        [('transformer', CountVectorizer(min_df=5, ngram_range=(1, 2))), ('estimator', get_mlp())]
    )

    x, y = get_data()
    scoring = {'accuracy': 'accuracy',
               'f1_macro': 'f1_macro',
               'precision_macro': 'precision_macro',
               'recall_macro' : 'recall_macro',
               'f1_weighted' : 'f1_weighted',
               'precision_weighted' : 'precision_weighted',
               'recall_weighted' : 'recall_weighted'
              }

    scores = cross_validate(pipeline, x, y, cv=5, scoring=scoring, n_jobs=-1)

    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))

    f1_weighted = np.mean(scores['test_f1_weighted'])
    precision_weighted = np.mean(scores['test_precision_weighted'])
    recall_weighted = np.mean(scores['test_recall_weighted'])
    print('F1 Weighted: ', f1_weighted)
    print('Precision Weighted: ', precision_weighted)
    print('Recall Weighted: ', recall_weighted)

if __name__ == '__main__':
    print('On Train/Test Split', end='\n\n')
    main()
    print('On K-fold Cross Validation', end='\n\n')
    k_fold()

On Train/Test Split

              precision    recall  f1-score   support

           0     0.7684    0.7019    0.7337       208
           1     0.6526    0.6814    0.6667       317
           2     0.6277    0.6396    0.6336       369
           3     0.6333    0.6244    0.6288       213

    accuracy                         0.6603      1107
   macro avg     0.6705    0.6618    0.6657      1107
weighted avg     0.6623    0.6603    0.6609      1107

F1 Weighted:  0.6609414283248674
Precision Weighted:  0.6623324392632884
Recall Weighted:  0.6603432700993677
On K-fold Cross Validation

Accuracy:  0.6559381283987644
F1 Macro:  0.658711231476486
Precision Macro:  0.669414684581496
Recall Macro:  0.6521219342334894
F1 Weighted:  0.6575132243730861
Precision Weighted:  0.6634191654404423
Recall Weighted:  0.6559381283987644


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier

# define the individual models to include in the ensemble
svm = get_svm()
mlp = get_mlp()
lr = get_logistic_regression()
rfc = get_random_forest()
gnb = get_naive_bayes()
x_train, x_test, y_train, y_test = get_train_test()

# define the ensemble model
ensemble = VotingClassifier(
    estimators=[('svm', svm), ('mlp', mlp), ('rfc', rfc), ('lr',lr), ('gnb',gnb)],
    voting='soft'
)

# use the ensemble model to fit and predict
ensemble.fit(x_train, y_train)
results = ensemble.predict(x_test)

# print the classification report
print(classification_report(y_test, results, digits=4))


# calculate additional evaluation metrics
accuracy = accuracy_score(y_test, results)
conf_matrix = confusion_matrix(y_test, results)
precision = precision_score(y_test, results, average='weighted')
recall = recall_score(y_test, results, average='weighted')
f1 = f1_score(y_test, results, average='weighted')

# print the evaluation metrics
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



              precision    recall  f1-score   support

           0     0.7797    0.6635    0.7169       208
           1     0.7129    0.6814    0.6968       317
           2     0.6170    0.7290    0.6683       369
           3     0.6754    0.6056    0.6386       213

    accuracy                         0.6793      1107
   macro avg     0.6962    0.6699    0.6801      1107
weighted avg     0.6862    0.6793    0.6799      1107

Accuracy: 0.6793134598012647
Confusion Matrix:
 [[138  14  48   8]
 [  7 216  67  27]
 [ 22  51 269  27]
 [ 10  22  52 129]]
Precision: 0.6862431547654728
Recall: 0.6793134598012647
F1 Score: 0.6798780851544699


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier

# define the individual models to include in the ensemble
svm = get_svm()
mlp = get_mlp()
lr = get_logistic_regression()
rfc = get_random_forest()
gnb = get_naive_bayes()
x_train, x_test, y_train, y_test = get_train_test()

# define the ensemble model
ensemble = VotingClassifier(
    estimators=[('mlp', mlp), ('lr',lr), ('gnb',gnb)],
    voting='soft'
)

# use the ensemble model to fit and predict
ensemble.fit(x_train, y_train)
results = ensemble.predict(x_test)

# print the classification report
print(classification_report(y_test, results, digits=4))


# calculate additional evaluation metrics
accuracy = accuracy_score(y_test, results)
conf_matrix = confusion_matrix(y_test, results)
precision = precision_score(y_test, results, average='weighted')
recall = recall_score(y_test, results, average='weighted')
f1 = f1_score(y_test, results, average='weighted')

# print the evaluation metrics
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



              precision    recall  f1-score   support

           0     0.7473    0.6683    0.7056       208
           1     0.6733    0.6372    0.6548       317
           2     0.5835    0.6721    0.6247       369
           3     0.6224    0.5728    0.5966       213

    accuracy                         0.6423      1107
   macro avg     0.6567    0.6376    0.6454      1107
weighted avg     0.6475    0.6423    0.6431      1107

Accuracy: 0.6422764227642277
Confusion Matrix:
 [[139  12  47  10]
 [ 11 202  76  28]
 [ 25  60 248  36]
 [ 11  26  54 122]]
Precision: 0.6475081413513624
Recall: 0.6422764227642277
F1 Score: 0.6430955576157813


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# split data into training and testing sets
X_train, X_test, y_train, y_test = get_train_test()

# create base classifier
base_classifier = DecisionTreeClassifier(max_depth=1, random_state=1)

# create boosting classifier
boosting_classifier = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=100, random_state=1)

# fit the boosting classifier on the training data
boosting_classifier.fit(X_train, y_train)

# make predictions on the test data
y_pred = boosting_classifier.predict(X_test)

# print classification report
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.46      0.56       208
           1       0.58      0.43      0.49       317
           2       0.46      0.70      0.56       369
           3       0.53      0.44      0.48       213

    accuracy                           0.53      1107
   macro avg       0.57      0.51      0.52      1107
weighted avg       0.55      0.53      0.52      1107

